In [0]:
%sql
CREATE CATALOG IF NOT EXISTS travel_agents;
CREATE SCHEMA IF NOT EXISTS train_agent;

In [0]:
%sql
CREATE OR REPLACE FUNCTION travel_agents.train_agent.get_connections(
    from_station STRING COMMENT 'The train station of departure', 
    to_station STRING COMMENT 'The train station of arrival',
    via_station STRING COMMENT 'The desired stops in between departure and arrival'
)
RETURNS STRING 
COMMENT 'Executes a call to the transport api and connections endpoint to retrieve relevant train connections given the input parameters from (departure), to (arrival), via (stops in between, if specified).'
LANGUAGE PYTHON
AS $$
import requests

url = "http://transport.opendata.ch/v1/connections"

params = {
    "from": from_station,
    "to": to_station,
    "via": via_station,
    "transportations": "train"
}

response = requests.get(url, params=params)

if response.status_code == 200:
    next_connection = response.json()
    return next_connection['connections']
else:
    return f"Failed to retrieve connection. Status code: {response.status_code}"
$$;

In [0]:
%sql
CREATE OR REPLACE FUNCTION travel_agents.train_agent.get_station_board(
    station STRING COMMENT 'The train station to retrieve the station board data', 
    arrival_or_departure STRING COMMENT 'Identifier if travel connections should depict arrival or depature'
)
RETURNS STRING 
COMMENT "Returns the station board data of a given train station to determine which train connections are leaving and arriving"
LANGUAGE PYTHON
AS $$
import requests

url = "http://transport.opendata.ch/v1/stationboard"
params = {
    "station": station,
    "type": arrival_or_departure,
    "limit": 15,
    "transportions": "train"
}

response = requests.get(url, params=params)

if response.status_code == 200:
    station_board = response.json()
    return station_board
else:
    return f"Failed to retrieve connection. Status code: {response.status_code}"
$$;